# Concept induction
This notebook offers sample EDSL code for using language models to identify concepts in unstructured texts, then generate criteria for the concepts, and then apply the criteria to evaluate the texts.

This idea is inspired by the recent paper <a href="https://hci.stanford.edu/publications/2024/Lam_LLooM_CHI24.pdf">Concept Induction: Analyzing Unstructured Text with High-Level Concepts Using LLooM</a>.

<a href="https://docs.expectedparrot.com/en/latest/">EDSL</a> is an open-source library for simulating surveys and experiments with AI agents and language models. Please see our <a href="https://docs.expectedparrot.com/en/latest/">documentation</a> for instructions on installing the package and <a href="https://discord.com/invite/mxAYkjfy9m">join our Discord</a> to ask questions and chat with other researchers!

## Identify concepts
We start by creating a general question prompting the respondent (a language model) to identify concepts in a given text that we will add later.

EDSL comes with a variety of <a href="https://docs.expectedparrot.com/en/latest/questions.html">question types</a> that we can choose from based on the form of the response that we want to get back from the model. `QuestionList` may be appropriate where we want the response to be formatted as a list of strings:

In [1]:
from edsl.questions import QuestionList

q_concepts = QuestionList(
    question_name = "concepts",
    question_text = "Identify the key concepts in the following text: {{ text }}",
    # max_list_items = # Optional 
)

We might want to ask some other questions about our data at the same time (a data labeling task). For example:

In [2]:
from edsl.questions import QuestionMultipleChoice

q_sentiment = QuestionMultipleChoice(
    question_name = "sentiment",
    question_text = "What is the sentiment of this text: {{ text }}?",
    question_options = ["Negative", "Neutral", "Positive"]
)

Then we parameterize the questions in order to run them for each of our texts. This is done with "scenario" objects that we create for our data (here, some recent tweets by Pres. Biden):

In [3]:
# Replace with your data
texts = [ # POTUS recent tweets
    "Tune in as I deliver the keynote address at the U.S. Holocaust Memorial Museum’s Annual Days of Remembrance ceremony in Washington, D.C.",
    "We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom.",
    "Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.",
    "Today, the Army Black Knights are taking home West Point’s 10th Commander-in-Chief Trophy. They should be proud. I’m proud of them too – not for the wins, but because after every game they hang up their uniforms and put on another: one representing the United States.",
    "This Holocaust Remembrance Day, we mourn the six million Jews who were killed by the Nazis during one of the darkest chapters in human history. And we recommit to heeding the lessons of the Shoah and realizing the responsibility of 'Never Again.'",
    "The recipients of the Presidential Medal of Freedom haven't just kept faith in freedom. They kept all of America's faith in a better tomorrow.",
    "Like Jill says, 'Teaching isn’t just a job. It’s a calling.' She knows that in her bones, and I know every educator who joined us at the White House for the first-ever Teacher State Dinner lives out that truth every day.",
    "Jill and I send warm wishes to Orthodox Christian communities around the world as they celebrate Easter. May the Lord bless and keep you this Easter Sunday and in the year ahead.",
    "Dreamers are our loved ones, nurses, teachers, and small business owners – they deserve the promise of health care just like all of us. Today, my Administration is making that real by expanding affordable health coverage through the Affordable Care Act to DACA recipients.",
    "With today’s report of 175,000 new jobs, the American comeback continues. Congressional Republicans are fighting to cut taxes for billionaires and let special interests rip folks off, I'm focused on job creation and building an economy that works for the families I grew up with."
]
len(texts)

10

In [4]:
from edsl import Scenario

scenarios = [Scenario({"text": text}) for text in texts]

Next we combine the questions into a survey in order to administer them together:

In [5]:
from edsl import Survey

survey = Survey(questions = [q_concepts, q_sentiment])

We add the scenarios to the survey and then run it to generate a dataset of results:

In [6]:
results = survey.by(scenarios).run()

EDSL comes with built-in methods for working with results in a variety of forms (data tables, SQL queries, dataframes, JSON, CSV). We can call the `columns` method to see a list of all the components that we can analyze:

In [7]:
results.columns

['agent.agent_name',
 'answer.concepts',
 'answer.sentiment',
 'comment.concepts_comment',
 'comment.sentiment_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.concepts_system_prompt',
 'prompt.concepts_user_prompt',
 'prompt.sentiment_system_prompt',
 'prompt.sentiment_user_prompt',
 'question_options.concepts_question_options',
 'question_options.sentiment_question_options',
 'question_text.concepts_question_text',
 'question_text.sentiment_question_text',
 'question_type.concepts_question_type',
 'question_type.sentiment_question_type',
 'raw_model_response.concepts_raw_model_response',
 'raw_model_response.sentiment_raw_model_response',
 'scenario.text']

We can select and print specific components to inspect:

In [8]:
results.select("text", "concepts", "sentiment").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ scenario                                        ┃ answer                                           ┃ answer     ┃
┃ .text                                           ┃ .concepts                                        ┃ .sentiment ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ Tune in as I deliver the keynote address at the │ ['keynote address', 'U.S. Holocaust Memorial     │ Neutral    │
│ U.S. Holocaust Memorial Museum’s Annual Days of │ Museum', 'Annual Days of Remembrance ceremony',  │            │
│ Remembrance ceremony in Washington, D.C.        │ 'Washington, D.C.']                              │            │
├─────────────────────────────────────────────────┼──────────────────────────────────────────────────┼────────────┤
│ We’re a nation of immigrants. A nation of       │ ['nation of immigrants', 'nation of dreamers',   │ Positive   │
│ dreamers. And as Cinco de Mayo represents, a    │ 'Cinco de Mayo', 'nation of freedom']            │            │
│ nation of freedom.                              │                                                  │            │
├─────────────────────────────────────────────────┼──────────────────────────────────────────────────┼────────────┤
│ Medicare is stronger and Social Security        │ ['Medicare', 'Social Security', 'economic plan', │ Positive   │
│ remains strong. My economic plan has helped     │ 'Medicare solvency', 'decade', 'extending Social │            │
│ extend Medicare solvency by a decade. And I am  │ Security solvency', 'rich', 'fair share']        │            │
│ committed to extending Social Security solvency │                                                  │            │
│ by making the rich pay their fair share.        │                                                  │            │
├─────────────────────────────────────────────────┼──────────────────────────────────────────────────┼────────────┤
│ Today, the Army Black Knights are taking home   │ ['Army Black Knights', 'West Point', '10th       │ Positive   │
│ West Point’s 10th Commander-in-Chief Trophy.    │ Commander-in-Chief Trophy', 'proud', 'uniforms', │            │
│ They should be proud. I’m proud of them too –   │ 'representing the United States']                │            │
│ not for the wins, but because after every game  │                                                  │            │
│ they hang up their uniforms and put on another: │                                                  │            │
│ one representing the United States.             │                                                  │            │
├─────────────────────────────────────────────────┼──────────────────────────────────────────────────┼────────────┤
│ This Holocaust Remembrance Day, we mourn the    │ ['Holocaust Remembrance Day', 'mourn', 'six      │ Negative   │
│ six million Jews who were killed by the Nazis   │ million Jews', 'killed', 'Nazis', 'darkest       │            │
│ during one of the darkest chapters in human     │ chapters in human history', 'heed', 'lessons',   │            │
│ history. And we recommit to heeding the lessons │ 'Shoah', 'responsibility', "'Never Again'"]      │            │
│ of the Shoah and realizing the responsibility   │                                                  │            │
│ of 'Never Again.'                               │                                                  │            │
├─────────────────────────────────────────────────┼──────────────────────────────────────────────────┼────────────┤
│ The recipients of the Presidential Medal of     │ ['recipients', 'Presidential Medal of Freedom',  │ Positive   │
│ Freedom haven't just kept faith in freedom.     │ 'faith in freedom', "America's faith", 'better   │            │
│ They kept all of America's faith in a better    │ tomo

If our concepts lists are too long, we can run another question prompting a model to condense it. We can specify the number of concepts that we want to get:

In [9]:
# Flattening our list of lists for all the texts to use in a follow-on question:
concepts_list = results.select("concepts").to_list(flatten=True)
# concepts_list

In [10]:
q_condense = QuestionList(
    question_name = "condense",
    question_text = "Return a condensed list of the following list of concepts: " + ", ".join(concepts_list),
    max_list_items = 10
)

Note that we can call the `run()` method on either a survey of questions or an individual question:

In [11]:
results = q_condense.run()

In [12]:
results.select("condense").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                                                                                                          ┃
┃ .condense                                                                                                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ['Holocaust Remembrance', 'Medicare and Social Security', 'Economic and Tax Plans', 'Military and National      │
│ Service', 'Educational Commitment', 'Cultural Celebrations', 'Healthcare Initiatives', 'Employment Growth',     │
│ 'Presidential Honors', 'National Messages']                                                                     │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

## Identify criteria for each concept
Similar to our first step, next we can run a question prompting the model to generate criteria for each concept. We could use `QuestionFreeText` generate these criteria in an unstructured narrative:

In [13]:
from edsl.questions import QuestionFreeText

q_criteria = QuestionFreeText(
    question_name = "criteria",
    question_text = """Describe key criteria for determining whether a text is primarily about the 
    following concept: {{ concept }}"""
)

For this question, the scenarios are the concepts that we generated:

In [14]:
condensed_concepts_list = results.select("condense").to_list(flatten=True)

scenarios = [Scenario({"concept": concept}) for concept in condensed_concepts_list]
scenarios

[Scenario({'concept': 'Holocaust Remembrance'}),
 Scenario({'concept': 'Medicare and Social Security'}),
 Scenario({'concept': 'Economic and Tax Plans'}),
 Scenario({'concept': 'Military and National Service'}),
 Scenario({'concept': 'Educational Commitment'}),
 Scenario({'concept': 'Cultural Celebrations'}),
 Scenario({'concept': 'Healthcare Initiatives'}),
 Scenario({'concept': 'Employment Growth'}),
 Scenario({'concept': 'Presidential Honors'}),
 Scenario({'concept': 'National Messages'})]

In [15]:
results = q_criteria.by(scenarios).run()

In [16]:
results.select("concept", "criteria").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ scenario                      ┃ answer                                                                          ┃
┃ .concept                      ┃ .criteria                                                                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Holocaust Remembrance         │ To determine if a text is primarily about Holocaust Remembrance, one would look │
│                               │ for several key criteria. Firstly, the text would likely mention specific       │
│                               │ events related to the Holocaust, such as the systematic persecution and         │
│                               │ annihilation of six million Jews by the Nazi regime. Secondly, it would discuss │
│                               │ acts of commemoration, such as memorial services, educational programs, and     │
│                               │ moments of silence. Thirdly, the text might reference symbols and observances   │
│                               │ associated with Holocaust Remembrance, like the date of International Holocaust │
│                               │ Remembrance Day (January 27th), the lighting of candles, or the reading of      │
│                               │ names of victims. Additionally, discussions of the importance of remembering    │
│                               │ the Holocaust to prevent future genocides, the role of survivors' testimonies,  │
│                               │ and the moral lessons learned from this dark period in history would be         │
│                               │ indicative of the text's focus on Holocaust Remembrance.                        │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ Medicare and Social Security  │ To determine if a text is primarily about Medicare and Social Security, one     │
│                               │ should look for the following key criteria: the presence of specific            │
│                               │ terminology related to Medicare (such as 'Part A', 'Part B', 'prescription drug │
│                               │ coverage', 'Medicare Advantage', 'Centers for Medicare & Medicaid Services')    │
│                               │ and Social Security ('Social Security benefits', 'retirement benefits',         │
│                               │ 'disability benefits', 'SSA', 'FICA'); a focus on the policies, regulations,    │
│                               │ reforms, and management of these programs; discussion of the impact on and      │
│                               │ eligibility criteria for beneficiaries; analysis of funding, sustainability,    │
│                               │ and financial aspects of the programs; and the inclusion of historical context  │
│                               │ and future projections related to Medicare and Social Security. If the majority │
│                               │ of these elements are present, it's likely that the text is primarily concerned │
│                               │ with these topics.                                                              │
├───────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│ Economic and Tax Plans        │ To determine whether a text is primarily about Economic and Tax Plans, key      │
│                               │ criteria would include the presence of discussions or information on fiscal     │
│                               │ policies, government budget allocations, taxation strategies, proposed changes  │
│                               │ to tax laws, economic growth initiatives, financial regulations, and investment │
│                               │ incentives. The text s

## Identify the concepts in each text and evaluate based on the criteria
Finally, we can use the concepts and the criteria to run another question where we prompt the model to evaulate each text. Question types `QuestionLinearScale`, `QuestionRank` or `QuestionNumerical` may be appropriate where we want to return a score:

In [17]:
from edsl.questions import QuestionLinearScale

q_score = QuestionLinearScale(
    question_name = "score",
    question_text = """Consider the following concept and criteria for determining whether 
    a given text addresses this concept. Then score how well the following text satisfies
    the criteria for the concept.
    Concept: {{ concept }}
    Criteria: {{ criteria }}
    Text: {{ text }}""",
    question_options = [0,1,2,3,4,5,6,7,8,9,10],
    option_labels = {0: "Not at all", 10: "Very well"} # Optional 
)

Here we want to use both the texts and the concepts and corresponding criteria together as scenarios of the question:

In [18]:
concepts_criteria = [list(pair) for pair in zip(results.select("concept").to_list(), results.select("criteria").to_list())]
len(concepts_criteria)

10

In [19]:
scenarios = [Scenario({"text": text, "concept": concept, "criteria": criteria}) 
             for text in texts 
             for [concept, criteria] in concepts_criteria]
len(scenarios)

100

In [20]:
results = q_score.by(scenarios).run()

We can filter the results based on the responses--e.g., here we just show the non-zero scores:

In [21]:
(results
 .filter("int(score) > 0")
 .select("text", "concept", "score")
 .print(format="rich")
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ scenario                                                               ┃ scenario                      ┃ answer ┃
┃ .text                                                                  ┃ .concept                      ┃ .score ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Tune in as I deliver the keynote address at the U.S. Holocaust         │ Holocaust Remembrance         │ 6      │
│ Memorial Museum’s Annual Days of Remembrance ceremony in Washington,   │                               │        │
│ D.C.                                                                   │                               │        │
├────────────────────────────────────────────────────────────────────────┼───────────────────────────────┼────────┤
│ Tune in as I deliver the keynote address at the U.S. Holocaust         │ National Messages             │ 1      │
│ Memorial Museum’s Annual Days of Remembrance ceremony in Washington,   │                               │        │
│ D.C.                                                                   │                               │        │
├────────────────────────────────────────────────────────────────────────┼───────────────────────────────┼────────┤
│ We’re a nation of immigrants. A nation of dreamers. And as Cinco de    │ Cultural Celebrations         │ 2      │
│ Mayo represents, a nation of freedom.                                  │                               │        │
├────────────────────────────────────────────────────────────────────────┼───────────────────────────────┼────────┤
│ We’re a nation of immigrants. A nation of dreamers. And as Cinco de    │ National Messages             │ 2      │
│ Mayo represents, a nation of freedom.                                  │                               │        │
├────────────────────────────────────────────────────────────────────────┼───────────────────────────────┼────────┤
│ Medicare is stronger and Social Security remains strong. My economic   │ Medicare and Social Security  │ 3      │
│ plan has helped extend Medicare solvency by a decade. And I am         │                               │        │
│ committed to extending Social Security solvency by making the rich pay │                               │        │
│ their fair share.                                                      │                               │        │
├────────────────────────────────────────────────────────────────────────┼───────────────────────────────┼────────┤
│ Medicare is stronger and Social Security remains strong. My economic   │ Economic and Tax Plans        │ 4      │
│ plan has helped extend Medicare solvency by a decade. And I am         │                               │        │
│ committed to extending Social Security solvency by making the rich pay │                               │        │
│ their fair share.                                                      │                               │        │
├────────────────────────────────────────────────────────────────────────┼───────────────────────────────┼────────┤
│ Medicare is stronger and Social Security remains strong. My economic   │ Healthcare Initiatives        │ 3      │
│ plan has helped extend Medicare solvency by a decade. And I am         │                               │        │
│ committed to extending Social Security solvency by making the rich pay │                               │        │
│ their fair share.                                                      │                               │        │
├────────────────────────────────────────────────────────────────────────┼───────────────────────────────┼────────┤
│ Medicare is stronger and Social Security remains strong. My economic   │ National Messages             │ 8      │
│ plan has helped extend Medicare solvency by a decade. 

Please see our <a href="https://docs.expectedparrot.com/en/latest/index.html">documentation page</a> for templates for other use cases and examples and <a href="https://discord.com/invite/mxAYkjfy9m">join our Discord</a> to ask questions and chat with other researchers!